# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

0. Make sure to select Input Materials
1. Execute "Run first: ..." cell below to load Input Materials into the current kernel
2. Set Input Parameters (e.g. `MILLER_INDICES`, `THICKNESS`, `MAX_AREA`) below or use the default values
3. Click "Run" > "Run All" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


<h2 style="color:red">Run first: load input materials in current kernel</h2>


In [30]:
import json
materials_in = []
for file in ["Ni.json", "Gr.json"]:
    with open(file, "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))

## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials

In [31]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
    "MILLER_INDICES": (1, 1, 2),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1, # the index of the material in the materials_in list
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

### 1.2. Set Interface Parameters

The distance between layer and substrate and maximum area to consider when matching.


In [32]:
INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0, # in Angstroms
    "MAX_AREA": 50, # in Angstroms^2
}

### 1.3. Set Algorithm Parameters

In [33]:
ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

## 2. Install Packages

In [34]:
# from jupyterlite.utils import install_packages

# await install_packages("create_interface_with_min_strain_zsl.ipynb")

## 3. Create interfaces

### 3.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.

In [35]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material, "\n")


def create_interfaces(settings):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
            termination,
            gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
            film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
            substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
            in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1,1,1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


interfaces, terminations = create_interfaces(
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)

Full Formula (Ni1)
Reduced Formula: Ni
abc   :   2.460000   2.460000   2.460000
angles:  60.000004  59.999994  60.000003
pbc   :       True       True       True
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Ni      0    0    0 

Full Formula (C2)
Reduced Formula: C
abc   :   2.467291   2.467291  20.000000
angles:  90.000000  90.000000 119.999986
pbc   :       True       True       True
Sites (2)
  #  SP           a         b    c
---  ----  --------  --------  ---
  0  C     0         0           0
  1  C     0.333333  0.666667    0 

Creating interfaces...


spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.750000
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.512500
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.286875
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.072531
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.868905
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.675459
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was 

### 3.2. Print out the interfaces and terminations

In [36]:
print(f'Found {len(terminations)} terminations')
for termination in terminations:
    print(f"Found {len(interfaces[termination])} interfaces for", termination, "termination")

Found 1 terminations
Found 3 interfaces for ('C_P6/mmm_2', 'Ni_Pmmm_1') termination


## 4. Sort interfaces by strain

### 4.1. Sort all interfaces

In [37]:
# Could be "strain", "von_mises_strain", "mean_abs_strain"
strain_mode = "mean_abs_strain"

# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


sorted_interfaces = sort_interfaces(interfaces, terminations)

### 4.2. Print out interfaces with lowest strain for each termination

In [38]:
for termination in terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    first_interface = interfaces[termination][0]
    print("    strain:", first_interface[strain_mode] * 100, "%")
    print("    number of atoms:", first_interface["interface"].num_sites)

Interface with lowest strain for termination ('C_P6/mmm_2', 'Ni_Pmmm_1') (index 0):
    strain: 0.23000000000000004 %
    number of atoms: 36


## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [39]:
import plotly.graph_objs as go
from collections import defaultdict

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}


def plot_strain_vs_atoms(sorted_interfaces, terminations, settings):
    # Create a mapping from termination to its index
    termination_to_index = {termination: i for i, termination in enumerate(terminations)}

    grouped_interfaces = defaultdict(list)
    for termination, interfaces in sorted_interfaces.items():
        for index, interface_data in enumerate(interfaces):
            strain_percentage = interface_data["mean_abs_strain"] * 100
            num_sites = interface_data["interface"].num_sites
            key = (strain_percentage, num_sites)
            grouped_interfaces[key].append((index, termination))
    
    data = []
    for (strain, num_sites), indices_and_terminations in grouped_interfaces.items():
        termination_indices = defaultdict(list)
        for index, termination in indices_and_terminations:
            termination_indices[termination].append(index)
        all_indices = [index for indices in termination_indices.values() for index in indices]
        index_range = f"{min(all_indices)}-{max(all_indices)}" if len(all_indices) > 1 else str(min(all_indices))
        
        hover_text = "<br>-----<br>".join(
             f"Termination: {termination}<br>Termination index: {termination_to_index[termination]}<br>Interfaces Index Range: {index_range}<br>Strain: {strain:.2f}%<br>Atoms: {num_sites}"
            for termination, indices in termination_indices.items()
        )
        trace = go.Scatter(
            x=[strain],
            y=[num_sites],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Indices: {index_range}",
        )
        data.append(trace)

    layout = go.Layout(
        xaxis=dict(title="Strain (%)", type=settings["X_SCALE"]),
        yaxis=dict(title="Number of atoms", type=settings["Y_SCALE"]),
        hovermode="closest",
        height=settings["HEIGHT"],
        legend_title_text="Interfaces Index Range",
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()



plot_strain_vs_atoms(sorted_interfaces, terminations, PLOT_SETTINGS)

for i, termination in enumerate(terminations):
    print(f"Termination {i}:", termination)

Termination 0: ('C_P6/mmm_2', 'Ni_Pmmm_1')


## 6. Select the interface to relax

### 6.1. Select the interface with the desired termination and strain

The data in `sorted_interfaces` now contains an object with the following structure:

```json
{
    "('C_P6/mmm_2', 'Si_R-3m_1')": [
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 0...},
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 1...},
        ...
    ],
    "<termination at index 1>": [
        { ...interface for 'termination at index 1' at index 0...},
        { ...interface for 'termination at index 1' at index 1...},
        ...
    ]
}
```

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [40]:
termination_index = 0
interface_index = 0

termination = terminations[termination_index]

interface = sorted_interfaces[termination][interface_index]["interface"]

## 7. Apply relaxation
### 7.1. Apply relaxation to the selected interface

In [41]:
# Per https://github.com/materialsvirtuallab/matgl/blob/main/examples/Relaxations%20and%20Simulations%20using%20the%20M3GNet%20Universal%20Potential.ipynb
from src.utils import poscar_to_ase, ase_to_poscar, ase_to_pymatgen, pymatgen_to_ase
from ase.optimize import BFGS
from ase.calculators.lj import LennardJones as LJ

# relax the interface
ase_interface = poscar_to_ase(interface.to(fmt="poscar"))
ase_interface.set_calculator(LJ())
dyn = BFGS(ase_interface)
dyn.run(fmax=0.01)

# extract results
ase_original_interface = pymatgen_to_ase(interface)
ase_relaxed_interface = ase_interface
relaxed_energy = ase_interface.get_total_energy()

# print out the final relaxed structure and energy
print('Original structure:\n', ase_to_poscar(ase_original_interface))
print('\nRelaxed structure:\n', ase_to_poscar(ase_relaxed_interface))
print(f"The final energy is {float(relaxed_energy):.3f} eV.")

      Step     Time          Energy         fmax
BFGS:    0 17:00:09      -11.324706        0.1384
BFGS:    1 17:00:09      -11.332162        0.1633
BFGS:    2 17:00:09      -11.438082        0.3872
BFGS:    3 17:00:09      -11.848308        0.7513
BFGS:    4 17:00:09      -13.314651        1.0419
BFGS:    5 17:00:09      -15.551924        0.1847
BFGS:    6 17:00:09      -15.578726        0.1916
BFGS:    7 17:00:09      -15.588771        0.1926
BFGS:    8 17:00:09      -15.724151        0.2215
BFGS:    9 17:00:09      -16.280454        0.4926
BFGS:   10 17:00:09      -18.473614        1.1777
BFGS:   11 17:00:09      -24.367547        2.5491
BFGS:   12 17:00:09       50.246599      142.9838
BFGS:   13 17:00:09      -24.575055        2.4819
BFGS:   14 17:00:09      -24.726183        2.4586
BFGS:   15 17:00:09      -20.550417       97.5342
BFGS:   16 17:00:09      -25.728115        2.1086
BFGS:   17 17:00:09      -25.933369        2.0235
BFGS:   18 17:00:09      -26.098170        1.7169
B

BFGS:   69 17:00:10      -39.556184       60.9746
BFGS:   70 17:00:10      -43.157249        4.0983
BFGS:   71 17:00:10      -40.661733       33.6850
BFGS:   72 17:00:10      -40.560012        4.6626
BFGS:   73 17:00:10      -43.253337        3.2947
BFGS:   74 17:00:10      -38.863907       89.9271
BFGS:   75 17:00:10      -44.113839        3.5139
BFGS:   76 17:00:10      -45.050002        4.4006
BFGS:   77 17:00:10      -38.452922      137.5509
BFGS:   78 17:00:10      -43.071188        6.3074
BFGS:   79 17:00:10      -45.166037        3.9328
BFGS:   80 17:00:10      -34.279462      195.0648
BFGS:   81 17:00:10      -45.787660       24.7740
BFGS:   82 17:00:10      -46.190383        4.1507
BFGS:   83 17:00:10      -47.717238        3.8678
BFGS:   84 17:00:10      -30.151939      120.3003
BFGS:   85 17:00:10      -49.541524        5.4584
BFGS:   86 17:00:10      -50.364700        3.4218
BFGS:   87 17:00:10      -51.285314        7.1383
BFGS:   88 17:00:10      -51.738432        4.7966


### 7.2. View structure before and after relaxation

In [42]:
from ase.visualize import view

# view(ase_original_interface, viewer="x3d")
# view(ase_relaxed_interface, viewer="x3d")


### 7.3. Calculate energy energy using ASE LJ

In [55]:
from ase import Atoms
nickel_relaxed_interface = ase_relaxed_interface.copy()
graphene_relaxed_interface = ase_relaxed_interface.copy()
nickel_original_interface = ase_original_interface.copy()
graphene_original_interface = ase_original_interface.copy()

del nickel_relaxed_interface[[atom.index for atom in nickel_relaxed_interface if atom.symbol != 'Ni']]
del graphene_relaxed_interface[[atom.index for atom in graphene_relaxed_interface if atom.symbol != 'C']]
del nickel_original_interface[[atom.index for atom in nickel_original_interface if atom.symbol != 'Ni']]
del graphene_original_interface[[atom.index for atom in graphene_original_interface if atom.symbol != 'C']]

#set LJ calculator
nickel_relaxed_interface.set_calculator(LJ())
graphene_relaxed_interface.set_calculator(LJ())
nickel_original_interface.set_calculator(LJ())
graphene_original_interface.set_calculator(LJ())

original_substrate_energy = nickel_original_interface.get_total_energy()
original_layer_energy = graphene_original_interface.get_total_energy()
relaxed_substrate_energy = nickel_relaxed_interface.get_total_energy()
relaxed_layer_energy = graphene_relaxed_interface.get_total_energy()

delta_original = relaxed_energy - original_substrate_energy - original_layer_energy
delta_relaxed = relaxed_energy - relaxed_substrate_energy - relaxed_layer_energy

# calculate area of interface
area = ase_original_interface.get_volume() / ase_original_interface.cell[2, 2]
print('Interface area:', area, 'Ang^2')
print()
effective_delta_relaxed = (relaxed_energy*ase_relaxed_interface.get_global_number_of_atoms() - 
                           nickel_relaxed_interface.get_global_number_of_atoms()*relaxed_substrate_energy - 
                           graphene_relaxed_interface.get_global_number_of_atoms()*relaxed_layer_energy)/(2*area*ase_relaxed_interface.get_global_number_of_atoms())

print(f"Relaxed interface energy: {relaxed_energy:.4f} eV")
print(f"Original Substrate energy: {original_substrate_energy:.4f} eV")
print(f"Original Layer energy: {original_layer_energy:.4f} eV")
print(f"Original Delta: {delta_original:.4f} eV")
print(f"Original Delta per area: {delta_original / area:.4f} eV/Ang^2")
print()

print(f"Relaxed Substrate energy: {relaxed_substrate_energy:.4f} eV")
print(f"Relaxed Layer energy: {relaxed_layer_energy:.4f} eV")
print(f"Relaxed Delta: {delta_relaxed:.4f} eV")
print(f"Relaxed Delta per area: {delta_relaxed / area:.4f} eV/Ang^2")
print(f"Effective relaxed Delta per area: {effective_delta_relaxed:.4f} eV/Ang^2")
print()

print(ase_to_poscar(ase_relaxed_interface))
print(ase_to_poscar(nickel_relaxed_interface))
print(ase_to_poscar(graphene_relaxed_interface))
print(ase_to_poscar(nickel_original_interface))
print(ase_to_poscar(graphene_original_interface))

Interface area: 42.79127281604002 Ang^2
Relaxed interface energy: -97.3516 eV
Original Substrate energy: -1.1239 eV
Original Layer energy: -10.2008 eV
Original Delta: -86.0269 eV
Original Delta per area: -2.0104 eV/Ang^2

Relaxed Substrate energy: -57.8833 eV
Relaxed Layer energy: -39.4684 eV
Relaxed Delta: 0.0000 eV
Relaxed Delta per area: 0.0000 eV/Ang^2
Effective relaxed Delta per area: -0.5568 eV/Ang^2

Ni  C 
 1.0000000000000000
     2.4600000686580152    0.0000000000000000    0.0000000000000002
     0.0000075375295471   17.3948258624982834    0.0000000000000011
     0.0000000000000000    0.0000000000000000   26.6899999999999977
 Ni  C  
  20  16
Cartesian
  1.1385800904901915 15.3883243872666657  1.4325991544198740
  1.5894977822256093  3.1762376860017913  1.7063446155199433
  0.8896408740622226  3.9647994846240153  1.3088135480937442
  1.0962343861699790  8.1873196150537773  1.8297014312916666
  1.7793581204915323 14.5359830648705000  1.1081837718973426
 -0.6814702878576279 -1.1

## 8. Pass relaxed interface to Materials Designer

In [ ]:
from jupyterlite.utils import set_data
from src.utils import from_pymatgen

esse_interface = from_pymatgen(ase_to_pymatgen(ase_relaxed_interface))
set_data("materials", [esse_interface] )

ImportError: This module intended to be used in a Pyodide environment. Please install packages yourself using pip.